In [ ]:
# IMPORT THE MODULES NEEDED
import os
import glob
import math
import datetime
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.python.keras.utils import losses_utils
#import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from sklearn.preprocessing import normalize
from classification_models.tfkeras import Classifiers

In [ ]:
## Define the parameters of the network and the save/log/data directories
# Change parameters here...
sizex = 500
sizey = 500
sizez = 3
rand_seed = 666
num_datasets = -1
batch_size = 32 * 4
nb_epochs = 400
validation_ratio = 0.2
ds_size = 405
buf_size = ds_size*2

# Parameter to also use the images with the dust rendered
use_dust = False

log_dir = "logs/"
os.makedirs(log_dir, exist_ok=True)
data_dir = 'data/'

# Define the desired resnet model
resnet_select = 'resnet18'
constrained_resnet = False

model_name = resnet_select + '_bs' + str(batch_size)
if use_dust: model_name += '_dust'

snapshot_weights = 'models/best_'+model_name+'.hdf5'
last_snapshot_weights = 'models/last_'+model_name+'.hdf5'
json_name = 'models/json'+model_name+'.json'
trained_model = 'models/trained'+model_name+'.h5'

In [ ]:
# Define the custom loss function from the cosine similarity impleted in tensorflow
class myCosineSimilarity(tf.keras.losses.Loss):
    def __init__(self,
               axis=-1,
               reduction=losses_utils.ReductionV2.AUTO,
               name='myCosineSimilarity'):
        super(myCosineSimilarity, self).__init__(reduction=reduction, name=name)
        self._axis = axis
    
    def call(self, y_true, y_pred):
        cossim = lambda y, p_y: -1*tf.keras.backend.abs(tf.keras.losses.cosine_similarity(y, p_y, axis=self._axis))
        return cossim(y_true, y_pred)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Defining the model in all the devices (GPU's) available
# import the correct resnet model
ResNet, preprocess_input = Classifiers.get(resnet_select)
with strategy.scope():
    # Build the model
    model = ResNet(input_shape=(sizex, sizey, sizez), include_top=False)

    if constrained_resnet:
        for i in range(88, 29, -1):
            model._layers.pop()

    globavg = tf.keras.layers.GlobalAveragePooling2D()(model.layers[-1].output)
    output = tf.keras.layers.Dense(2, activation=None)(globavg)
    model = tf.keras.models.Model(inputs=model.inputs, outputs=output)
    
    # Use existing best weights if available...
    if os.path.isfile(last_snapshot_weights):
        print('loaded weights')
        model.load_weights(last_snapshot_weights)
    
    # Compile the model specifying the optimazer (sgd) and the custom loss function and other metrics
    model.compile(optimizer= tf.keras.optimizers.SGD(),
                  loss= myCosineSimilarity(), metrics=[tf.keras.losses.cosine_similarity,
                                                       tf.keras.metrics.mean_absolute_error,])

In [ ]:
# See a sumary of the model with all the layers and parameters
model.summary()

In [ ]:
# Collect all the .npz files
numpy_datasets = glob.glob(data_dir+'dataset*')

# load the data into a dictionary from npz files
data_dict = {}
for dataset in numpy_datasets[:num_datasets]:
    with np.load(dataset) as data:
        if 'y' in data_dict.keys():
            data_dict['x_dust']   = np.append( data_dict['x_dust'], data['x_dust'],axis=0)
            data_dict['x_nodust'] = np.append( data_dict['x_nodust'], data['x_nodust'],axis=0)
            data_dict['y']        = np.append( data_dict['y'], data['y'],axis=0)
            data_dict['ids']      = np.append( data_dict['ids'], data['id'],axis=0)
        else:
            data_dict['x_dust']   = data['x_dust']
            data_dict['x_nodust'] = data['x_nodust']
            data_dict['y']        = data['y']
            data_dict['ids']      = data['id']

In [ ]:
# check the shape of the dictionary (100 images of 500x500x3 = 4 files of 25 images) before filtering
print(data_dict['x_nodust'].shape)

In [ ]:
#Filter the galaxies that have less than 1e10 solar masses
bad_ids = [[galaxy_id,i] for galaxy_id,i in zip(data_dict['ids'],range(len(data_dict['ids']))) if 'e09' in galaxy_id or 'e10' in galaxy_id ]
bad_ids = np.array(bad_ids)
remove_galaxies = np.array(bad_ids[:,1],dtype=int)
print('there are ', len(bad_ids), ' galaxies to remove')
#print(remove_galaxies)
filtered_dict = {}

for key in data_dict.keys():
    data_dict[key] = np.delete(data_dict[key],remove_galaxies, axis=0)
    
print('Still have ', len(data_dict['ids']), ' galaxies left')
# check that we have filtered the correct galaxies
print(data_dict['ids'][:5])
# check the shape of the dictionary after filtering
print("shape of image's array",data_dict['x_dust'].shape)
# check the shape of our dictionary in te labels entry
print("shape of the label's array",data_dict['y'].shape)

In [ ]:
# computing the normaliced vectors in 3D
data_dict['y_norm'] = normalize(data_dict['y'], axis=1)

In [ ]:
# Functions to do the augmentation
def random_flip_lr(image, label, seed=None):
    img = tf.image.random_flip_left_right(image, seed=seed)
    if tf.reduce_all(tf.equal(img,image)):
        label_f = label
    else:
        label_f = tf.convert_to_tensor([-label[0],label[1]])
    return (img, label_f)

def random_flip_ud(image, label, seed=None):
    img = tf.image.random_flip_up_down(image, seed=seed)
    if tf.reduce_all(tf.equal(img,image)):
        label_f = label
    else:
        label_f = tf.convert_to_tensor([label[0],-label[1]])
    return (img,label_f)


def random_rot(image, label, seed=None):
    number_of_intervalls = 24 # 15 deg steps
    
    rad = tf.random.uniform(shape=[1], minval=0, maxval=2*math.pi, dtype=tf.float32)
    rad = rad//(2*math.pi/number_of_intervalls) * (2*math.pi/number_of_intervalls)
    #rad = tf.math.multiply(deg, tf.constant(math.pi/180, dtype=tf.float32))
    img = tfa.image.rotate(image, rad, interpolation = 'BILINEAR')
    
    x = tf.math.multiply(tf.math.cos(-rad), label[0]) - tf.math.multiply(tf.math.sin(-rad), label[1])
    y = tf.math.multiply(tf.math.sin(-rad), label[0]) + tf.math.multiply(tf.math.cos(-rad), label[1])
    label_f = tf.reshape(tf.convert_to_tensor([x, y]), [2])
    return (img,label_f)

def augment(img, label):
    img_f,label_f = random_flip_lr(img,label)
    img_f,label_f = random_flip_ud(img_f,label_f)
    img_f,label_f = random_rot(img_f,label_f)
    return (img_f, label_f)

In [ ]:
# Import the autotune option from TF
AUTO = tf.data.experimental.AUTOTUNE

# Defining the train and validation dataset from the existing dictionary, shufling and batching
# First we do the ones without dust, then if duts is activated we make it also with dust and mix both
input_ds_nodust = (
    tf.data.Dataset.from_tensor_slices((data_dict['x_nodust'].astype(np.float32),
                                        data_dict['y_norm'][:,:2].astype(np.float32)))
                    # Use seed to ensure we always have the same validation data set!
                   .shuffle(ds_size, seed = rand_seed, reshuffle_each_iteration = False))

# take a part of the dataset (we take all but there's the option to take just a part of it with the ds_size parameter)
input_ds_nodust = input_ds_nodust.take(ds_size)
# compute the val_size dataset in terms of the ds_size and the validation_ratio that we have
val_size = math.floor(ds_size * validation_ratio)

# cache and augment the dataset (THE ORDER IS IMPORTANT!!)
validation_ds_nodust_aug =  (input_ds_nodust.take(val_size).cache()
                            .map(augment, num_parallel_calls=AUTO))
train_ds_nodust_aug      =  (input_ds_nodust.skip(val_size).cache()
                            .map(augment, num_parallel_calls=AUTO))

# if dust is activated compute the dust datasets
if use_dust:
    input_ds_dust = (
        tf.data.Dataset.from_tensor_slices((data_dict['x_dust'].astype(np.float32),
                                            data_dict['y_norm'][:,:2].astype(np.float32)))
                       .shuffle(ds_size, seed = rand_seed, reshuffle_each_iteration = False))
    
    input_ds_dust = input_ds_dust.take(ds_size)
    validation_ds_dust_aug = (input_ds_dust.take(val_size).cache()
                              .map(augment, num_parallel_calls=AUTO))
    train_ds_dust_aug =  (input_ds_dust.skip(val_size).cache()
                          .map(augment, num_parallel_calls=AUTO))
    
    # concatenate both datasets (dust and no dust) and repeat the data and batch it
    train_ds = (train_ds_dust_aug.concatenate(train_ds_nodust_aug)
                    .shuffle(buffer_size=buf_size, reshuffle_each_iteration=True)
                    .repeat(48).batch(batch_size).prefetch(AUTO))
    validation_ds = (validation_ds_dust_aug.concatenate(validation_ds_nodust_aug)
                    .repeat(48).batch(batch_size).prefetch(AUTO))

else:
    # if dust is not activated compute the dataset from the nodust dataset and reapeat the data and batch it
    train_ds = (train_ds_nodust_aug
                    .shuffle(buffer_size=buf_size, reshuffle_each_iteration=True)
                    .repeat(48).batch(batch_size).prefetch(AUTO))
    validation_ds = (validation_ds_nodust_aug
                    .repeat(48).batch(batch_size).prefetch(AUTO))

# check all the datasets to see if they are correct
print(validation_ds_nodust_aug)
print(train_ds_nodust_aug)
print(validation_ds)
print(train_ds)

In [ ]:
# function to visualice the data 
def viz(img, label, pred=np.array([0,0,0]), id=None):
    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(img, cmap='gray', vmin=0, vmax=255,origin='lower')
    ax.arrow(250,250,100*label[0],100*label[1], head_width=7, head_length=10, fc='g', ec='g')
    if np.sum(np.abs(pred)) > 0: ax.arrow(250,250,100*pred[0],100*pred[1], head_width=7, head_length=10, fc='r', ec='r')
        
    display(Markdown("#### **cartesian label:** {}".format(label)))
    if np.sum(np.abs(pred)) > 0: display(Markdown("#### **cartesian prediction (x_y):** {}".format(pred)))
    if id: display(Markdown("#### **ID:** {}".format(id)))
    plt.show()

# visualice the data
for idx in range(1,5):
    elements = train_ds.take(idx)
    for elem in elements:
        viz(elem[0][idx], elem[1][idx])#, elem['y_revs'][idx], id=elem['ids'][idx])

In [ ]:
# Launch tensorboard
%reload_ext tensorboard
%tensorboard --logdir {log_dir} --reload_multifile True --host 0.0.0.0 --port 6006

In [ ]:
# serialize model to JSON for latter infernce
model_json = model.to_json()
with open(json_name, "w") as json_file:
    json_file.write(model_json)

In [ ]:
# For tensorboard to save the models
log_dir_s = log_dir + model_name
os.makedirs(log_dir_s, exist_ok=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_s, histogram_freq=1, update_freq='batch', profile_batch=0)

#define the learning rate scheduler
def scheduler(epoch, lr):
    if epoch == 200:
        return lr #/ 100
    else:
        return lr

# Checkpointing
checkpointer_1 = tf.keras.callbacks.ModelCheckpoint(filepath=snapshot_weights,
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True)
checkpointer_2 = tf.keras.callbacks.ModelCheckpoint(filepath=last_snapshot_weights,
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=False)
LearningRateScheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [
    tensorboard_callback,
    checkpointer_1,
    checkpointer_2,
    LearningRateScheduler,
]

# Fit the model with the training and validation data, number of epochs and callbacks
model.fit(
    train_ds,
    validation_data = validation_ds,
    epochs = nb_epochs,
    callbacks = callbacks,)

In [ ]:
#save the last weights of the model
model.save(trained_model)

In [ ]:
# taking samples from the validation dataset and evaluating the model
val_ex, = validation_ds.take(1)
train_ex, = train_ds.take(1)

images_v, labels_v = val_ex[0], val_ex[1]
images_t, labels_t = train_ex[0], train_ex[1]

#making the predictions for the taken images
outputs_v = model.predict(images_v)
outputs_t = model.predict(images_t)

In [ ]:
# visualicing the results
for img,label,pred,i in zip(images_t, labels_t, outputs_t,range(5)):
    viz(img, label, pred)

In [ ]:
for img,label,pred,i in zip(images_v, labels_v, outputs_v,range(5)):
    viz(img, label, pred)